In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "clientes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_cliente"
TABELA_INVALIDOS_DESTINO = "tb_cliente_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

EMAIL_REGEX = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ShortType # Garantindo os imports necessários

cliente_bronze_df = spark.read.table(nome_tabela_origem)

# --- INÍCIO DA LÓGICA DE FILTRO POR CARGA MAIS RECENTE ---
# 1. Encontrar a data/timestamp de ingestão mais recente
max_dt_ingestion = (
    cliente_bronze_df
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

# 2. Filtrar o DataFrame Bronze apenas pela carga mais recente
df_filtrado = cliente_bronze_df.filter(
    F.col("ingestion_timestamp") == F.lit(max_dt_ingestion)
)
# --- FIM DA LÓGICA DE FILTRO POR CARGA MAIS RECENTE ---

df_transformado = (
    # Usar o DataFrame filtrado
    df_filtrado
    .select(
        F.substring(F.sha2(F.col("id_cliente").cast(StringType()), 256), 1, 36).alias("cd_cliente"),
        F.trim(F.col("nome")).alias("nm_cliente"),
        # Correção no regexp_replace: a expressão original r'\+[A-Za-z0-9._%+-]+@'
        # parece querer remover o "plus addressing" (ex: email+tag@dominio.com -> email@dominio.com)
        F.regexp_replace(F.col("email"), r'\+[^@]+@', "@").alias("ds_email"),
        F.trim(F.col("regiao")).alias("nm_regiao"),
        F.col("idade").cast(ShortType()).alias("nu_idade"),
        F.col("ingestion_timestamp").alias("dt_ingestion"), 
        F.col("origem").alias("dc_origem")
    )
    # Deduplicação feita APENAS dentro da carga mais recente
    .dropDuplicates(["cd_cliente"])
)

# O df_transformado agora contém apenas a carga mais recente e deduplicada.

In [0]:
df_validacao = (
    df_transformado
    .withColumn("flag_email_valido", F.col("ds_email").rlike(EMAIL_REGEX))
    .withColumn("flag_maior_idade", F.col("nu_idade") >= 18)
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_email_valido") & 
            F.col("flag_maior_idade"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

df_silver = df_validos.select(
    "cd_cliente",
    "nm_cliente",
    "ds_email",
    "nm_regiao",
    "nu_idade",
    "dt_ingestion",
    "dc_origem"
)

display(df_silver)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_cliente = origem.cd_cliente"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")
print(f"Total de registros válidos processados: {df_silver.count()}")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria!")
print(f"Total de registros inválidos: {df_invalidos.count()}")